In [1]:
# %load std_libraries.py
#to keep my scripts consistent, I am adding this to the top of all scripts with %load std_libraries.py
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn import linear_model
#import pickle
#import subprocess
#import networkx as nx
#import scipy.stats as stats
#import statsmodels.graphics.gofplots as stats_graph
#import scipy.cluster.hierarchy as sch

# from Bio import SeqIO
# from Bio import SeqFeature as sf
# from Bio.Alphabet import generic_dna
# from Bio.Seq import Seq

import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02_student','9dMTMZgJMgUP0YX0P5mQ')
    #py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')
    
# for phylogenetic trees: 
# from ete3 import Tree

#for scraping internet data (e.g. ncbi)
#import requests
#from lxml import etree    #parses xml output

what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
Added C:\Users\heine\github\expression_broad_data to path: 
['', 'C:\\Users\\heine\\Anaconda3\\python36.zip', 'C:\\Users\\heine\\Anaconda3\\DLLs', 'C:\\Users\\heine\\Anaconda3\\lib', 'C:\\Users\\heine\\Anaconda3', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\heine\\.ipython', 'C:\\Users\\heine\\github\\expression_broad_data']
I am about to import a library
I am importing io_library
what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
data process

In [2]:
# Parse original data file and make CSV for raw expression and perturbations
# Only need to do this the first time you use the data set.  Subsequent runs can read data directly from the CSV. 
# Species List: S. Cerevisiae, K. Lactis, S. Bayanus, C. Glabrata, N. Castelli

#Did these in 20150118_SB_SC_CG_plots
#['Saccharomyces cerevisiae', 'Kluyveromyces lactis', 'Candida glabrata', 'Saccharomyces castellii' , 'Saccharomyces bayanus']
#spec_abbrev = ['Scer', 'Klac', 'Cgla', 'Ncas','Sbay']
species_list = ['Lacancea kluyverii']#['Saccharomyces mikatae','Lachancea waltii','Saccharomyces paradoxus','Lacancea kluyverii','Debaryomyces hansenii','Vanderwaltozyma polyspora']
### here
spec_abbrev = ['Lklu'] #['Smik', 'Lwal','Spar','Lklu','Dhan','Vpol']
dir_out = data_processing_dir + os.path.normpath('regev_data/')
io_library.make_data_tables(species_list,spec_abbrev, dir_out)


Species: Lacancea kluyverii, Line 10170 was blank, made nan
Species: Lacancea kluyverii, Line 10429 was blank, made nan
Species: Lacancea kluyverii, Line 10629 was blank, made nan
Species: Lacancea kluyverii, Line 10744 was blank, made nan
Species: Lacancea kluyverii, Line 11246 was blank, made nan
Species: Lacancea kluyverii, Line 11347 was blank, made nan
Species: Lacancea kluyverii, Line 11439 was blank, made nan
Species: Lacancea kluyverii, Line 11440 was blank, made nan
Species: Lacancea kluyverii, Line 11529 was blank, made nan
Species: Lacancea kluyverii, Line 11573 was blank, made nan
Species: Lacancea kluyverii, Line 11574 was blank, made nan
Species: Lacancea kluyverii, Line 11895 was blank, made nan
Species: Lacancea kluyverii, Line 11943 was blank, made nan
Species: Lacancea kluyverii, Line 11945 was blank, made nan
Species: Lacancea kluyverii, Line 12071 was blank, made nan
Species: Lacancea kluyverii, Line 12076 was blank, made nan
Species: Lacancea kluyverii, Line 12128 

In [11]:
#Load gois
goi_data = pd.read_csv(data_processing_dir + os.sep + "20170817_klscpka" + os.sep + "goi_data.csv") 

#get expression data for S.Cer
## should make into function

#Load stress and growth data
scer_growth_stress = io_library.combine_growth_stress_datasets('Scer')
scer_growth_stress.drop('ID', axis = 1, inplace=True)
scer_growth_stress.rename(columns = dict(zip(scer_growth_stress.columns, [column + '_Scer' for column in scer_growth_stress.columns])),inplace=True)

#Merge growth and stress Data
goi_data_cross_spec_exp = goi_data.merge(scer_growth_stress, how = 'left', left_on= 'sc_genename', right_index = True)

#Merge raw exp data
spec = 'Scer'
fname = data_processing_dir + os.path.normpath("regev_data/raw_exp/"  + spec + '_raw_exp.csv')
raw_exp = pd.read_csv(fname, index_col = 'orf_name')

goi_data_cross_spec_exp = goi_data_cross_spec_exp.merge(pd.DataFrame(raw_exp['Mean']),left_on = 'sc_genename', right_index = True, how = 'left')
goi_data_cross_spec_exp.rename(columns = {'Mean': 'raw_'+spec}, inplace = True)

species = ['Spar','Smik','Sbay', 'Cgla', 'Ncas','Klac','Lwal','Lklu','Dhan']  
#About half the Dhan gene orthologs don't match up to the data. 
#Vpol is not here because it doesn't have an ortholog mapping
for spec in species: 
    
    #load ortholog table
    if spec=='Klac': #use YGOB for Klac
        orth_dir = data_processing_dir + 'ortholog_files_YGOB' + os.sep
    else:
        orth_dir = data_processing_dir + 'ortholog_files' + os.sep

    orth_lookup = io_library.read_orth_lookup_table('Scer', spec, orth_dir)

    #determine correct orthologs for all gois
    spec_genes = []
    corr_sc_genes = []
    for sc_gene in goi_data['sc_genename']:
        if sc_gene in orth_lookup.keys():
            spec_gene = orth_lookup[sc_gene]
            if len(spec_gene)>1:
                print(sc_gene + ' has more than one ortholog in ' + spec)
                for spec_gene_jj in spec_gene: 
                    spec_genes.append(spec_gene_jj)
                    corr_sc_genes.append(sc_gene)
            elif len(spec_gene)==1:
                if spec_gene[0]=='NONE':
                    print(sc_gene + ' has no orthologs in ' + spec)
                else:
                    spec_genes.append(spec_gene[0])
                    corr_sc_genes.append(sc_gene)
        else: 
            print(sc_gene + ' not present in ortholog map ' + spec)

    #Load raw_exp data
    fname = data_processing_dir + os.path.normpath("regev_data/raw_exp/"  + spec + '_raw_exp.csv')
    raw_exp = pd.read_csv(fname, index_col = 'orf_name')

    #make subest of gois and make sc_genename column

    #remove CG genes that are missing from the array
    missing_from_array = set(spec_genes)-set(raw_exp.index)
    if len(missing_from_array)>0:
        drop_ind = [ind for ind,gene in enumerate(spec_genes) if gene in missing_from_array]
        print('The following ' + spec + ' genes do not appear in the array: ')
        for ind in drop_ind: 
            print(spec_genes[ind] + ', sc_orth ' + corr_sc_genes[ind])

        corr_sc_genes = [sc_gene for ind, sc_gene in enumerate(corr_sc_genes) if not(ind in drop_ind)]
        spec_genes = [gene for ind, gene in enumerate(spec_genes) if not(ind in drop_ind)]


    raw_exp_subset = raw_exp.loc[spec_genes,:].copy()
    raw_exp_subset['sc_genename'] = corr_sc_genes
    raw_exp_subset.reset_index(inplace=True)

    #combine mean raw data with avg microarray data

    #Growth experiment
    fname = os.path.normpath(data_processing_dir + "regev_data/GSE36253_Growth/"  + spec + '_growth.csv' )
    growth_exp = pd.read_csv(fname,header = [0,1,2], index_col = [0,1])
    print(fname + ' growth microarray dataset loaded')

    #group by conditions and take mean
    growth_replicate_groups = growth_exp.groupby(axis = 1, level = 'conditions')
    growth_exp_avg = growth_replicate_groups.aggregate(np.mean)
    growth_exp_avg.reset_index(inplace=True)
    growth_exp_avg.drop(columns = 'ID', inplace = True)

    spec_data_subset = raw_exp_subset.loc[:,['orf_name','Mean','sc_genename']].merge(growth_exp_avg, on = 'orf_name', how = 'left' )
    column_rename = {'Mean' : 'raw_'+spec }
    growth_conds = list(growth_exp_avg.columns.drop('orf_name'))
    for cond in growth_conds: 
        column_rename[cond] = cond + '_' + spec

    spec_data_subset.rename(columns = column_rename, inplace=True)

    #stress 
    if spec in {'Klac','Ncas','Cgla','Lwal'}:
        fname = os.path.normpath(data_processing_dir + "regev_data/GSE38478_Stress/"  + spec + '_stress.csv' )
        stress_exp = pd.read_csv(fname,header = [0,1,2], index_col = [0,1])
        print(fname + ' stress microarray dataset loaded')

        #group by condition and take mean
        stress_replicate_groups = stress_exp.groupby(axis = 1, level = 'conditions')
        stress_exp_avg = stress_replicate_groups.aggregate(np.mean)
        stress_exp_avg.reset_index(inplace=True)
        stress_exp_avg.drop(columns = 'ID', inplace = True)

        spec_data_subset = spec_data_subset.merge(stress_exp_avg, on = 'orf_name', how = 'left' )

        stress_conds = list(stress_exp_avg.columns.drop('orf_name'))
        column_rename = {}
        for cond in stress_conds: 
            column_rename[cond] = cond + '_' + spec
        spec_data_subset.rename(columns = column_rename, inplace=True)

    spec_data_subset.rename(columns = {'orf_name': spec + '_genename'}, inplace=True)

    goi_data_cross_spec_exp = goi_data_cross_spec_exp.merge(spec_data_subset,on = 'sc_genename', how = 'outer')

goi_data_cross_spec_exp.to_csv(data_processing_dir + os.sep + "20170817_klscpka" + os.sep + "goi_data_cross_spec_expt.csv")

C:\Users\heine\github\expression_broad_data\expression_data\regev_data\GSE36253_Growth\Scer_growth.csv growth microarray dataset loaded
C:\Users\heine\github\expression_broad_data\expression_data\regev_data\GSE38478_Stress\Scer_stress.csv stress microarray dataset loaded
combined dataset saved as C:\Users\heine\github\expression_broad_data\expression_data\regev_data\Scer_growth_stress.csv
YHR183W has no orthologs in Spar
YJL196C has no orthologs in Spar
YNR034W has no orthologs in Spar
YJL045W has no orthologs in Spar
YLR164W has no orthologs in Spar
YPR009W has no orthologs in Spar
YOL131W has no orthologs in Spar
YDR385W has more than one ortholog in Spar
YOR133W has no orthologs in Spar
YLR029C has more than one ortholog in Spar
YMR121C has no orthologs in Spar
YGR131W has no orthologs in Spar
YGR138C has no orthologs in Spar
YBL043W has no orthologs in Spar
YJR115W has no orthologs in Spar
YDR034W-B has no orthologs in Spar
YBR056W-A not present in ortholog map Spar
YDR312W has no 

The following Klac genes do not appear in the array: 
KLLA0E20263g, sc_orth YBL061C
KLLA0E20263g, sc_orth YER096W
KLLA0E15467g, sc_orth YGR239C
KLLA0E15467g, sc_orth YHR160C
KLLA0E12123g, sc_orth YKL141W
KLLA0E12123g, sc_orth YMR118C
KLLA0E21891g, sc_orth YDL085W
KLLA0E21891g, sc_orth YMR145C
KLLA0E11265g, sc_orth YML100W
KLLA0E11265g, sc_orth YMR261C
KLLA0E05545g, sc_orth YBL039C
KLLA0E05545g, sc_orth YJR103W
KLLA0E01057g, sc_orth YDR077W
KLLA0E01057g, sc_orth YER150W
KLLA0E23981g, sc_orth YBR183W
KLLA0E23981g, sc_orth YPL087W
KLLA0E20373g, sc_orth YBL068W
KLLA0E20373g, sc_orth YER099C
KLLA0E06557g, sc_orth YBL101C
KLLA0E06557g, sc_orth YPR030W
KLLA0E03081g, sc_orth YDR389W
KLLA0E03081g, sc_orth YOR134W
KLLA0E04049g, sc_orth YDR144C
KLLA0E04049g, sc_orth YLR120C
KLLA0E07283g, sc_orth YNL309W
KLLA0E07283g, sc_orth YOL131W
KLLA0E16875g, sc_orth YDR259C
KLLA0E16875g, sc_orth YOR028C
KLLA0E19933g, sc_orth YBL054W
KLLA0E19933g, sc_orth YER088C
KLLA0E12563g, sc_orth YBR082C
KLLA0E12563g, sc

DEHA0E02618g, sc_orth YMR181C
DEHA0E02618g, sc_orth YPL229W
DEHA0D16918g, sc_orth YKR075C
DEHA0D16918g, sc_orth YOR062C
DEHA0E08074g, sc_orth YDR312W
DEHA0E08074g, sc_orth YHR066W
DEHA0A09790g, sc_orth YMR175W
DEHA0A09790g, sc_orth YPL223C
DEHA0G15730g, sc_orth YDR358W
DEHA0G15730g, sc_orth YHR108W
DEHA0A13310g, sc_orth YER045C
DEHA0A13310g, sc_orth YIL036W
DEHA0E22968g, sc_orth YGL031C
DEHA0E22968g, sc_orth YGR148C
DEHA0D05434g, sc_orth YEL047C
DEHA0D05434g, sc_orth YJR051W
DEHA0C06930g, sc_orth YIR018W
DEHA0C06930g, sc_orth YOL028C
DEHA0B08052g, sc_orth YHR206W
DEHA0B08052g, sc_orth YJR147W
DEHA0D02376g, sc_orth YBR118W
DEHA0D02376g, sc_orth YPR080W
DEHA0A13200g, sc_orth YIL133C
DEHA0A13200g, sc_orth YNL069C
DEHA0E23628g, sc_orth YCL043C
DEHA0E23628g, sc_orth YDR518W
DEHA0B05412g, sc_orth YFL034C-A
DEHA0B05412g, sc_orth YLR061W
C:\Users\heine\github\expression_broad_data\expression_data\regev_data\GSE36253_Growth\Dhan_growth.csv growth microarray dataset loaded


,Unnamed: 0,SC_common_name,sc_genename,log2FoldChange_SC,raw_AS_noNMPP1_YPD_t40_mean_SC,pvalue_SC,raw_AS_yesNMPP1_YPD_t40_mean_SC,kl_genename,log2FoldChange_KL,raw_AS_noNMPP1_YPD_t40_mean_KL,...,PLAT/LOG_Lklu,PS/LOG_Lklu,Dhan_genename,raw_Dhan,ELL/LOG_Dhan,LAG/LOG_Dhan,LL/LOG_Dhan,LPS/LOG_Dhan,PLAT/LOG_Dhan,PS/LOG_Dhan
0,3956,YMC2,YBR104W,-4.971720,2.686396,1.340605e-36,1.356552,KLLA0B08503g,-0.523493,2.199667,...,-0.902871,-1.056706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3957,YMC1,YPR058W,-0.995521,2.495893,1.912123e-03,2.323990,KLLA0B08503g,-0.523493,2.199667,...,-0.902871,-1.056706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3962,PHO87,YCR037C,-0.196522,2.638901,5.180891e-01,2.717109,KLLA0C03454g,-0.838158,3.019175,...,0.009994,-0.723324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3963,PHO90,YJL198W,-2.928895,2.945628,2.790440e-22,2.198321,KLLA0C03454g,-0.838158,3.019175,...,0.009994,-0.723324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3968,GLK1,YCL040W,4.160390,1.791886,3.592258e-04,3.224448,KLLA0C01155g,2.178913,2.557021,...,6.533852,5.934990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3969,EMI2,YDR516C,2.464130,3.271367,1.357405e-16,4.148126,KLLA0C01155g,2.178913,2.557021,...,6.533852,5.934990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3972,SKT5,YBL061C,-1.806729,2.460149,3.018311e-07,2.056057,KLLA0E20263g,-0.380750,2.462343,...,0.437645,-0.173988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3973,SHC1,YER096W,3.793451,1.471903,2.452936e-16,2.754438,KLLA0E20263g,-0.380750,2.462343,...,0.437645,-0.173988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3978,GND2,YGR256W,7.143522,1.412570,3.201154e-118,3.676099,KLLA0A09339g,0.394234,3.570254,...,-2.063340,-0.410460,DEHA0D06160g,-1.308734,2.690329,-2.378877,2.617460,2.471163,2.257091,2.468521
9,3979,GND1,YHR183W,-1.190240,2.855015,1.105005e-02,2.639284,KLLA0A09339g,0.394234,3.570254,...,-2.063340,-0.410460,DEHA0D06160g,-1.308734,2.690329,-2.378877,2.617460,2.471163,2.257091,2.468521


In [2]:
#Load goi data with expression
goi_data_cross_spec_exp = pd.read_csv(data_processing_dir + os.sep + "20170817_klscpka" + os.sep + "goi_data_cross_spec_expt.csv", low_memory=False)



In [6]:
list(goi_data_cross_spec_exp.columns)



['Unnamed: 0',
 'Unnamed: 0.1',
 'SC_common_name',
 'sc_genename',
 'log2FoldChange_SC',
 'raw_AS_noNMPP1_YPD_t40_mean_SC',
 'pvalue_SC',
 'raw_AS_yesNMPP1_YPD_t40_mean_SC',
 'kl_genename',
 'log2FoldChange_KL',
 'raw_AS_noNMPP1_YPD_t40_mean_KL',
 'raw_AS_yesNMPP1_YPD_t40_mean_KL',
 'pvalue_KL',
 'category',
 'SGD_description',
 'Go Slim Terms',
 'DS/LOG_Scer',
 'LAG/LOG_Scer',
 'LL/LOG_Scer',
 'PLAT/LOG_Scer',
 'PS/LOG_Scer',
 'NaCl_005_Scer',
 'NaCl_015_Scer',
 'NaCl_030_Scer',
 'NaCl_060_Scer',
 'heat shock_005_Scer',
 'heat shock_015_Scer',
 'heat shock_030_Scer',
 'heat shock_045_Scer',
 'heat shock_060_Scer',
 'hydrogen peroxide_005_Scer',
 'hydrogen peroxide_015_Scer',
 'hydrogen peroxide_030_Scer',
 'hydrogen peroxide_060_Scer',
 'raw_Scer',
 'Spar_genename',
 'raw_Spar',
 'DS/LOG_Spar',
 'LAG/LOG_Spar',
 'LPS/LOG_Spar',
 'PLAT/LOG_Spar',
 'PS/LOG_Spar',
 'Smik_genename',
 'raw_Smik',
 'DS/LOG_Smik',
 'LAG/LOG_Smik',
 'LL/LOG_Smik',
 'PLAT/LOG_Smik',
 'PS/LOG_Smik',
 'Sbay_gene

In [11]:
gene[condition + '_' + pka_exp_conv[spec]].values[0]

-1.19024014438568

In [9]:
#edit cox5a and cox5b

goi_subset = ['GND1','GND2']

gene = 'SOL4'

goi_data_row = goi_data_cross_spec_exp[goi_data_cross_spec_exp['SC_common_name']==gene]

conditions = ['log2FoldChange','PS/LOG','NaCl_015','heat shock_045','hydrogen peroxide_060','raw_AS_noNMPP1_YPD_t40_mean','raw']

pka_exp_conv = {'Scer':'SC', 'Klac':'KL'}

spec_order = ['Scer','Spar','Smik','Sbay', 'Cgla', 'Ncas','Klac','Lwal','Lklu','Dhan']

gene_table = pd.DataFrame(index = conditions)

for spec in spec_order: 
    column_spec = []
    for condition in conditions: 
        if condition in {'log2FoldChange','raw_AS_noNMPP1_YPD_t40_mean'}: 
            if spec in {'Scer','Klac'}:
                column_spec.append(goi_data_row[condition + '_' + pka_exp_conv[spec]].values[0])
            else: 
                column_spec.append(None)
        else: 
            column_name = condition + '_' + spec
            if column_name in goi_data_row.keys():
                column_spec.append(goi_data_row[column_name].values[0])
            else:
                column_spec.append(None)
    gene_table[spec] = column_spec
    
gene_table.to_csv(data_processing_dir + os.sep + "20170817_klscpka" + os.sep + "cross_spec_exp_" + gene + ".csv")
    

In [10]:
gene_table

,Scer,Spar,Smik,Sbay,Cgla,Ncas,Klac,Lwal,Lklu,Dhan
log2FoldChange,7.194901,NaN,NaN,NaN,NaN,NaN,1.734949,NaN,NaN,NaN
PS/LOG,9.185190,7.774536,4.422456,4.946814,NaN,2.292331,0.503764,NaN,0.988885,0.000000
NaCl_015,8.046359,NaN,NaN,NaN,NaN,0.430969,0.394378,NaN,NaN,NaN
heat shock_045,7.998886,NaN,NaN,NaN,NaN,2.088639,1.055003,NaN,NaN,NaN
hydrogen peroxide_060,2.683871,NaN,NaN,NaN,NaN,0.499373,-0.424343,NaN,NaN,NaN
raw_AS_noNMPP1_YPD_t40_mean,1.497447,NaN,NaN,NaN,NaN,NaN,2.541098,NaN,NaN,NaN
raw,-6.115741,-5.525725,-6.241377,-4.250805,NaN,-5.940816,-0.104851,NaN,-0.604952,-1.446374
